# Projeto Udacity blablabla

# Data Wraling

### 1. Gather 

In [1]:
#bibliotecas necessárias
import pandas as pd
import numpy as np
import requests
import tweepy
import json
from datetime import datetime as dt

#### 1.1 CSV

buscar da planilha



In [2]:
df_wrd = pd.read_csv('twitter-archive-enhanced.csv') #dataframe we rate dogs

#### 1.2 TSV

buscar arquivo da url



In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
rows  = response.text.split('\n')
#deleta última linha em branco
del rows[len(rows)-1]

for i in range(len(rows)):
    rows[i] = rows[i].split('\t')
header = rows[0]
#deleta header
del rows[0]
df_prediction = pd.DataFrame(np.array(rows), columns=header)



#### 1.3 Twitter

buscar informações na API e colocar no arquivo tweet_json.txt


In [4]:
# Pretty Print JSON method
def pp_json(json_thing, sort=True, indents=4):
    if type(json_thing) is str:
        print(json.dumps(json.loads(json_thing), sort_keys=sort, indent=indents))
    else:
        print(json.dumps(json_thing, sort_keys=sort, indent=indents))
    return None


def default_json(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError
    
def getInfoFromTwitter(df_wrd, consumerkey, consumer_secret, access_token, access_token_secret):

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    ids = df_wrd.tweet_id
    tweet_info = {}
    erros_count = 0
    print("Início: ", dt.now().time())
    for i in ids:    
        try:
            status = api.get_status(i) 
            tweet_info[i] = {'favorite_count': status.favorite_count, 'retweet_count':status.retweet_count }           
        except tweepy.TweepError as error:
            print("Erro.. ", error, " ", dt.now().time())        
            erros_count +=1        
    print("Total: ", len(ids))
    print("Erros: ", erros_count)
    print("Fim: ", dt.now().time())
    #write json
    with open('tweet_json.txt', 'w') as outfile:  
        json.dump(tweet_info, outfile, default=default_json)



In [5]:
consumer_key = 'hXCuMDzF2drKbyZpbX9v1iAJy'
consumer_secret = 'tGg83j5aekyYhrEjktbALmgFHSH9fjrmt4hzjv1HidHjmJMQQJ'

access_token = '955172750846496768-wOYuhJwjJhZc8IN4bme7pYU3v05soGi'
access_token_secret = 'vH8jf9lNDYtAAwQ7IgsEsxwmBfkdoGptg6bwstrQVM7ZP'

#getInfoFromTwitter(df_wrd, consumer_key, consumer_secret, access_token, access_token_secret)


In [6]:
#open json file
with open('tweet_json.txt') as file:
    twitter_data = json.load(file)

data = []
for key in twitter_data.keys():
    data.append([key, twitter_data[key]['favorite_count'], twitter_data[key]['retweet_count']])
header = ['twitter_id', 'favorite_count', 'retweet_count']
df_twitter = pd.DataFrame(np.array(data), columns=header)


### 2. Access



In [ ]:
#df_wrd, df_prediction, df_twitter

In [15]:
df_wrd.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1229,713761197720473600,NaN,NaN,2016-03-26 16:15:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Piper. She would really like that tenn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713761197...,12,10,Piper,None,None,None,None
1238,712097430750289920,NaN,NaN,2016-03-22 02:03:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I can't even comprehend how confused this dog ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/712097430...,10,10,None,None,None,None,None
377,828011680017821696,NaN,NaN,2017-02-04 22:45:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Brutus and Jersey. They think the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/828011680...,11,10,Brutus,None,None,None,None
656,791780927877898241,NaN,NaN,2016-10-27 23:17:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Maddie. She gets some w...,7.467577e+17,4.196984e+09,2016-06-25 17:31:25 +0000,"https://vine.co/v/5BYq6hmrEI3,https://vine.co/...",11,10,Maddie,None,None,None,None
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,20,16,None,None,None,None,None


In [52]:
df_wrd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [7]:
df_wrd.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_wrd[df_wrd.in_reply_to_status_id.notna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


In [18]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1620,802952499103731712,https://pbs.twimg.com/media/CySpCSHXcAAN-qC.jpg,1,chow,0.944032,True,golden_retriever,0.0172398,True,Pomeranian,0.0120848,True
39,666649482315059201,https://pbs.twimg.com/media/CUBqKnLWwAA5OQB.jpg,1,Border_collie,0.44780299999999995,True,English_springer,0.170497,True,collie,0.139206,True
305,671520732782923777,https://pbs.twimg.com/media/CVG4i9UWEAAUH3U.jpg,1,Pomeranian,0.551031,True,Pekinese,0.135262,True,gibbon,0.061557400000000005,False
1436,773922284943896577,https://pbs.twimg.com/media/Cr2GNdlW8AAbojw.jpg,1,Pomeranian,0.554331,True,Samoyed,0.432158,True,chow,0.00319942,True
1946,862457590147678208,https://pbs.twimg.com/media/C_gQmaTUMAAPYSS.jpg,1,home_theater,0.49634799999999996,False,studio_couch,0.167256,False,barber_chair,0.052625,False
1770,827600520311402496,https://pbs.twimg.com/media/C3w6RYbWQAAEQ25.jpg,1,Pembroke,0.32563800000000004,True,golden_retriever,0.317235,True,Labrador_retriever,0.116087,True
12,666073100786774016,https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg,1,Walker_hound,0.260857,True,English_foxhound,0.17538199999999998,True,Ibizan_hound,0.0974705,True
158,668872652652679168,https://pbs.twimg.com/media/CUhQIAhXAAA2j7u.jpg,1,teddy,0.413379,False,pillow,0.325623,False,miniature_schnauzer,0.0355366,True
1761,826115272272650244,https://pbs.twimg.com/media/C3bzVILWcAUjS5i.jpg,1,tennis_ball,0.9970709999999999,False,golden_retriever,0.00233085,True,kuvasz,0.000283472,True
245,670468609693655041,https://pbs.twimg.com/media/CU37pEoWUAAitje.jpg,1,minivan,0.730152,False,beach_wagon,0.07866139999999999,False,car_wheel,0.06434580000000001,False


In [54]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null object
p1          2075 non-null object
p1_conf     2075 non-null object
p1_dog      2075 non-null object
p2          2075 non-null object
p2_conf     2075 non-null object
p2_dog      2075 non-null object
p3          2075 non-null object
p3_conf     2075 non-null object
p3_dog      2075 non-null object
dtypes: object(12)
memory usage: 194.6+ KB


In [19]:
df_prediction.describe()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
count,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075
unique,2075,2009,4,378,2006,2,405,2004,2,408,2006,2
top,762316489655476224,https://pbs.twimg.com/media/CsGnz64WYAEIDHJ.jpg,1,golden_retriever,0.427742,True,Labrador_retriever,0.0693617,True,Labrador_retriever,0.07143619999999999,True
freq,1,2,1780,150,2,1532,104,3,1553,79,2,1499


#### 2.1 Problemas de Qualidade


##### Tabela We Rate Dogs
- in_reply_to_status_id  é float e  deveria ser int
- in_reply_to_user_id  é float e deveria ser int
- retweeted_status_id same
- retweeted_status_user_id   same 
- timestamp deveria ser data
- retweeted_status_timestamp     deveriam ser datas
- source deveria conter apenas a url
- rating_numerator possui outliers
- rating_denominator possui grandes outliers
- expanded url nulls values null

##### Tabela Predição

- tweet_id deveria ser int64
- img_num deveria ser int
- p1_dog deveria ser boolean
- p2_dog deveria ser boolean
- p3_dog deveria ser boolean
- p2_conf deveria ser float
- p1_conf deveria ser float
- p3_conf deveria ser float
- problema com maiuscula e minuscula nas raças
- problema com espaço e _ nas raças







#### 2.2 Problemas de Arrumação

##### Tabela We Rate Dogs

- Quatro ultimas colunas deveriam ser apenas uma
- Nome da coluna timestamp, retweeted_status_timestamp

##### Tabela Predição
- nome de todas as colunas 


### 3. Clean